

<img src="ALMAARC.png" width="250" height="240"  align="right"/> 



<hr style="border:2px solid darkblue"> </hr>

# NB2. Query a catalogue

<hr style="border:2px solid darkblue"> </hr>

In this notebook two functions are presented which query a list of sources from a catalogue: 

* By name
* By coordinate

-------------

The relevant columns in the ALMA Archive datasets are 
* *s_ra*, *s_dec* (RAD and DEC in degress)
* *target_name* (source name)
 
--------- ----- -----





In [1]:
from astropy.table import Table
import pandas as pd
pd.set_option('display.max_columns', None)
import numpy as np
import pyvo

service = pyvo.dal.TAPService("https://almascience.eso.org/tap")


<hr style="border:2px solid darkblue"> </hr>


## Query list of names
<hr style="border:2px solid darkblue"> </hr>



In [2]:
def query_name_list(name_list):
    
    query_strs=[]
    
    for i in range(len(name_list)):
        if i==0:
            query = f"""  SELECT  *  
                          FROM  ivoa.obscore  
                          WHERE target_name = '{name_list[i]}'
                     """
            query_strs.append(query)
        else:
            query = f""" target_name = '{name_list[i]}'"""
            
            query_strs.append(query)

    query_catalog= " OR ".join(query_strs)
    
    return service.search(query_catalog).to_table().to_pandas()

<hr style="border:2px solid darkblue"> </hr>

## Query list of coordinates

<hr style="border:2px solid darkblue"> </hr>



In [3]:

def query_coord_list(RASs, DECs, radius):

    query_strs=[]
    
    for i in range(len(RASs)):
        if i==0:
            query = f"""
                     SELECT  DISTANCE( POINT('ICRS', s_ra, s_dec),
                                       POINT('ICRS','{RAs[i]}','{DECs[i]}'))
                     AS dist, *
                     FROM ivoa.obscore
                     WHERE CONTAINS( POINT('ICRS', s_ra, s_dec),
                                     CIRCLE('ICRS','{RAs[i]}','{DECs[i]}' ,{radius}))=1
                     """
            query_strs.append(query)
        else:
            query = f"""
                     CONTAINS(
                     POINT('ICRS', s_ra, s_dec),
                     CIRCLE('ICRS','{RAs[i]}','{DECs[i]}' ,{radius}))=1
                     """
            query_strs.append(query)

    query_catalog= "OR".join(query_strs) 
    
    return service.search(query_catalog).to_table().to_pandas()

## Example 2a: Query a list of sources

In [4]:
source_list = ['CenA', 'M83', 'ALESS73.1'] 
results = query_name_list(source_list)

results.head(5)

,access_url,access_format,proposal_id,data_rights,gal_longitude,gal_latitude,obs_publisher_did,obs_collection,facility_name,instrument_name,obs_id,dataproduct_type,calib_level,target_name,s_ra,s_dec,s_fov,s_region,s_resolution,t_min,t_max,t_exptime,t_resolution,em_min,em_max,em_res_power,pol_states,o_ucd,band_list,em_resolution,authors,pub_abstract,publication_year,proposal_abstract,schedblock_name,proposal_authors,sensitivity_10kms,cont_sensitivity_bandwidth,pwv,group_ous_uid,member_ous_uid,asdm_uid,obs_title,type,scan_intent,science_observation,spatial_scale_max,bandwidth,antenna_arrays,is_mosaic,obs_release_date,spatial_resolution,frequency_support,frequency,velocity_resolution,obs_creator_name,pub_title,first_author,qa2_passed,bib_reference,science_keyword,scientific_category,lastModified
0,http://almascience.org/aq?member_ous_id=uid://...,text/html,2011.0.00010.S,Public,309.515914,19.417224,ADS/JAO.ALMA#2011.0.00010.S,ALMA,JAO,ALMA,uid://A002/X327408/X217,cube,2,CenA,201.365063,-43.019112,0.007385,Circle ICRS 201.365063 -43.019112 0.003692,1.330912,55950.253573,55950.305961,1512.000,1512.000,0.001364,0.001369,450286.77208,/XX/YY/,phot.flux.density;phys.polarization,6,614.023326,"McCoy, Mark; Ott, Jürgen; Meier, David S.; Mul...","Centaurus A, with its gas-rich elliptical host...",2017,Centaurus A with its host NGC5128 is the most ...,"CenA 13CO, C18O, HNCO, H2CO COMP","Impellizzeri, Violette; Peck, Alison; Walter, ...",1.399871,0.061695,0.852892,,uid://A002/X327408/X217,uid://A002/X383b50/Xc45,The Physics and Chemisty of Gas in Centaurus A...,S,TARGET,T,12.599759,937500000.0,A002:PM02 A004:DV04 A009:DA43 A011:DV12 A013:D...,F,2015-02-12T13:48:59.000,1.330912,"[217.59..218.53GHz,488.28kHz,1.4mJy/beam@10km/...",219.025328,663.940713,"Ott, Juergen",ALMA Observations of the Physical and Chemical...,"McCoy, Mark",T,2017ApJ...851...76M,"Active Galactic Nuclei (AGN)/Quasars (QSO), Me...",Active galaxies,2021-08-09T17:58:23.481
1,http://almascience.org/aq?member_ous_id=uid://...,text/html,2011.0.00010.S,Public,309.515914,19.417224,ADS/JAO.ALMA#2011.0.00010.S,ALMA,JAO,ALMA,uid://A002/X327408/X217,cube,2,CenA,201.365063,-43.019112,0.007385,Circle ICRS 201.365063 -43.019112 0.003692,1.330912,55950.253573,55950.305961,1512.000,1512.000,0.001365,0.001371,449784.96600,/XX/YY/,phot.flux.density;phys.polarization,6,614.023326,"McCoy, Mark; Ott, Jürgen; Meier, David S.; Mul...","Centaurus A, with its gas-rich elliptical host...",2017,Centaurus A with its host NGC5128 is the most ...,"CenA 13CO, C18O, HNCO, H2CO COMP","Impellizzeri, Violette; Peck, Alison; Walter, ...",1.400488,0.061695,0.852892,,uid://A002/X327408/X217,uid://A002/X383b50/Xc45,The Physics and Chemisty of Gas in Centaurus A...,S,TARGET,T,12.599759,937500000.0,A002:PM02 A004:DV04 A009:DA43 A011:DV12 A013:D...,F,2015-02-12T13:48:59.000,1.330912,"[217.59..218.53GHz,488.28kHz,1.4mJy/beam@10km/...",219.025328,663.940713,"Ott, Juergen",ALMA Observations of the Physical and Chemical...,"McCoy, Mark",T,2017ApJ...851...76M,"Active Galactic Nuclei (AGN)/Quasars (QSO), Me...",Active galaxies,2021-08-09T17:58:23.481
2,http://almascience.org/aq?member_ous_id=uid://...,text/html,2011.0.00772.S,Public,314.600374,31.972870,ADS/JAO.ALMA#2011.0.00772.S,ALMA,JAO,ALMA,uid://A002/X3216af/X31,cube,2,M83,204.269845,-29.862594,0.054640,Polygon ICRS 204.258830 -29.886906 204.254315 ...,1.140462,56013.300683,56166.036243,757.304,757.304,0.002637,0.002659,232808.19800,/XX/YY/,phot.flux.density;phys.polarization,3,614.003552,"Egusa, Fumi; Hirota, Akihiko; Baba, Junichi; M...",We describe an archival project using Cycle 0 ...,2018,Molecular-rich galaxies with prominent galacti...,m83-CO(1-0)-com m83-CO(1-0)-ext,"Muraoka, Kazuyuki; Nakanishi, Kouichiro; Ishiz...",7.805726,0.278101,1.821328,,uid://A002/X3216af/X31,uid://A002/X3b3400/X90f,Giant Molecular Cloud Survey Toward bar and ar...,S,TARGET,T,11.275626,937500000.0,A003:DA41 A004:DV04 A008:DV19 A011:DV12 A015:D...,T,2013-09-28T08:06:00.000,1.140462,"

-----------------

## Example 2b: Query a catalogue of high-z quasars

First we import a catalogue of high redshift quasars from Vizier, using astroquery:

In [13]:
from astroquery.vizier import Vizier
catalog_list = Vizier.get_catalogs('J/ApJS/227/11')
highzqso_catalog = catalog_list['J/ApJS/227/11/highzqso']


We will now use the RA and DEC coordinates in the catalogue to do the search in the ALMA archive. To start with a shorter search we will only search for the first 50 sources.

In [18]:
RAs=highzqso_catalog['RAJ2000'][0:30]
DECs=highzqso_catalog['DEJ2000'][0:30]

We now query the ALMA archive:


In [ ]:
results = query_coord_list(RAs, DECs, 0.01)


In [ ]:
results.head(5) ### show only 5, for visualization purposes

We can now plot the distribution of ALMA observed quasars from the chosen catalogue.

In [ ]:
import matplotlib.pyplot as plt 

plt.rcParams["figure.figsize"] = (15,10)

plt.plot(RAs,DECs, ls='', marker='.', ms=9, label='all')
plt.plot(results['s_ra'],results['s_dec'], ls='', marker='o',ms=12, fillstyle='none', label='observed by ALMA')
plt.legend()